# Cell Tracking and Lineage Construction in Live-Cell Imaging Data

### Part 1 (of 2)

## Training a Model
---

Implementation of:

[Accurate cell tracking and lineage construction in live-cell imaging experiments with deep learning](https://www.biorxiv.org/content/10.1101/803205v2)

Deployed at:

[DeepCell.org](http://www.deepcell.org/)

### Global Imports

In [1]:
import os
import datetime
import errno
import argparse

import numpy as np

import deepcell

### Load the Training Data

In [2]:
# Training a tracking algorithm is a complicated process that requires alot of data
# We recommend combining multiple data sets  

# Download four different sets of data (saves to ~/.keras/datasets)
filename_3T3 = '3T3_NIH.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.nih_3t3.load_tracked_data(filename_3T3)
print('3T3 -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

filename_HeLa = 'HeLa_S3.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.hela_s3.load_tracked_data(filename_HeLa)
print('HeLa -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

filename_HEK = 'HEK293.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.hek293.load_tracked_data(filename_HEK)
print('HEK293 -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

filename_RAW = 'RAW2647.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.raw2647.load_tracked_data(filename_RAW)
print('RAW264.7 -\nX.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

3T3 -
X.shape: (192, 30, 154, 182, 1)
y.shape: (192, 30, 154, 182, 1)
HeLa -
X.shape: (144, 40, 216, 256, 1)
y.shape: (144, 40, 216, 256, 1)
HEK293 -
X.shape: (207, 30, 135, 160, 1)
y.shape: (207, 30, 135, 160, 1)
RAW264.7 -
X.shape: (99, 30, 202, 240, 1)
y.shape: (99, 30, 202, 240, 1)


In [3]:
# Compile multiple TRKS files into one

# NB: .trks files are a special format that includes image and lineage data in np arrays
# To access, use the following utils
from deepcell.utils.tracking_utils import load_trks
from deepcell.utils.tracking_utils import save_trks

# Define a normalizaiton function for the raw images that can be run before padding
def image_norm(original_image):
    # NNs prefer input data that is 0 mean and unit variance
    normed_image = (original_image - np.mean(original_image)) / np.std(original_image)
    return normed_image

# Define all the trks to load
basepath = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))
trks_files = [os.path.join(basepath, filename_3T3), 
              os.path.join(basepath, filename_HeLa), 
              os.path.join(basepath, filename_HEK),
              os.path.join(basepath, filename_RAW)]

# Each TRKS file may have differrent dimensions but the model expects uniform dimensions
# So we will determine max dimensions and zero pad as neccesary
max_frames = 1
max_y = 1
max_x = 1

for trks_file in trks_files:
    trks = load_trks(trks_file)

    # Store dimensions of raw and tracked to check new data against to pad if neccesary
    if trks['X'][0].shape[0] > max_frames:
        max_frames = trks['X'][0].shape[0]
    if trks['X'][0].shape[1] > max_y:
        max_y = trks['X'][0].shape[1]
    if trks['X'][0].shape[2] > max_x:
        max_x = trks['X'][0].shape[2]

# Load each trks file, normalize and pad as neccesary
lineages = []
X = []
y = []        

k = 0
movie_counter = 0
for trks_file in trks_files:
    trks = load_trks(trks_file)
    for i, (lineage, raw, tracked) in enumerate(zip(trks['lineages'], trks['X'], trks['y'])):
        movie_counter = k + i

        # Normalize the raw images
        for frame in range(raw.shape[0]):
            raw[frame, :, :, 0] = image_norm(raw[frame, :, :, 0]) 
            
        # Image padding if neccesary - This assumes that raw and tracked have the same shape
        if raw.shape[1] < max_y:
            diff2pad = max_y-raw.shape[1]
            pad_width = int(diff2pad/2)
            if diff2pad % 2 == 0:
                # Pad width can be split evenly
                raw = np.pad(raw, ((0,0), (pad_width,pad_width), (0,0), (0,0)), mode='constant', constant_values=0)
                tracked = np.pad(tracked, ((0,0), (pad_width,pad_width), (0,0), (0,0)), mode='constant', constant_values=0)
            else:
                # Pad width cannot be split evenly
                raw = np.pad(raw, ((0,0), (pad_width+1,pad_width), (0,0), (0,0)), mode='constant', constant_values=0)
                tracked = np.pad(tracked, ((0,0), (pad_width+1,pad_width), (0,0), (0,0)), mode='constant', constant_values=0)

        if raw.shape[2] < max_x:
            diff2pad = max_x-raw.shape[2]
            pad_width = int(diff2pad/2)
            if diff2pad % 2 == 0:
                # Pad width can be split evenly
                raw = np.pad(raw, ((0,0), (0,0), (pad_width,pad_width), (0,0)), mode='constant', constant_values=0)
                tracked = np.pad(tracked, ((0,0), (0,0), (pad_width,pad_width), (0,0)), mode='constant', constant_values=0)
            else:
                # Pad width cannot be split evenly
                raw = np.pad(raw, ((0,0), (0,0), (pad_width+1,pad_width), (0,0)), mode='constant', constant_values=0)
                tracked = np.pad(tracked, ((0,0), (0,0), (pad_width+1,pad_width), (0,0)), mode='constant', constant_values=0)
        
        if raw.shape[0] < max_frames:   
            pad_width = int(max_frames-raw.shape[0])
            raw = np.pad(raw, ((0,pad_width), (0,0), (0,0), (0,0)), mode='constant', constant_values=0)
            tracked = np.pad(tracked, ((0,pad_width), (0,0), (0,0), (0,0)), mode='constant', constant_values=0)
        
        lineages.append(lineage)
        X.append(raw)
        y.append(tracked)
                
    k = movie_counter + 1

# Save the combined datasets into one trks file
filename = 'combined_data.trks'
save_trks(os.path.join(basepath, filename), lineages, X, y)

# View stats on this combined file
from deepcell.utils.tracking_utils import trks_stats
trks_stats(os.path.join(basepath, filename))

Dataset Statistics: 
Image data shape:  (803, 40, 216, 256, 1)
Number of lineages (should equal batch size):  803
Total number of unique tracks (cells)      -  12697
Total number of divisions                  -  944
Average cell density (cells/100 sq pixels) -  0.017033540852301552
Average number of frames per track         -  25


In [15]:
# combined_data.trks contains all of the data available

# To hold out a portion of this data for testing we will establish a random seed
test_seed = 1

# And how much of the data to hold out
test_size = .1

# Get the full dataset
trks = load_trks(os.path.join(basepath, filename))
total_data_size = trks['X'].shape[0]

# Select a portion of this dataset randomly 
import random
random.seed(test_seed)
train_data_range = int(total_data_size*(1-test_size))

idx_train = random.sample(range(total_data_size), train_data_range)

lineages, X, y = [], [], []
for i in idx_train:
    lineages.append(trks['lineages'][i])
    X.append(trks['X'][i])
    y.append(trks['y'][i])       

# Resave the portion we wish to use as the training (and validation) dataset
filename_train = 'combined_training_data.trks'
save_trks(os.path.join(basepath, filename_train), lineages, X, y)

# View stats on this combined file
trks_stats(os.path.join(basepath, filename_train))

Dataset Statistics: 
Image data shape:  (722, 40, 216, 256, 1)
Number of lineages (should equal batch size):  722
Total number of unique tracks (cells)      -  11510
Total number of divisions                  -  844
Average cell density (cells/100 sq pixels) -  0.017189596498441827
Average number of frames per track         -  25


### Set up filepath constants

In [16]:
# The path to the data file is currently required for `train_model_()` functions

# Change DATA_DIR if you are not using `deepcell.datasets`
DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))

# DATA_FILE should be a trks file (contains 2 np arrays and a lineage dictionary)
DATA_FILE = os.path.join(DATA_DIR, filename_train)

# confirm the data file is available
assert os.path.isfile(DATA_FILE)

In [17]:
# Set up other required filepaths

# If the data file is in a subdirectory, mirror it in MODEL_DIR and LOG_DIR
PREFIX = os.path.relpath(os.path.dirname(DATA_FILE), DATA_DIR)

ROOT_DIR = '/data'  # TODO: Change this! Usually a mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

### Training a New Model

#### Set up training parameters

In [19]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

n_epoch = 10     # Number of training epochs
test_size = .20  # % of data saved as validation
train_seed = 1   # Random seed for training/validation data split

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# Tracking training settings
features = {'appearance', 'distance', 'neighborhood', 'regionprop'}
min_track_length = 9
neighborhood_scale_size=30
batch_size = 128  
crop_dim = 32
in_shape = (crop_dim, crop_dim, 1)

tracking_model_name = 'tracking_model_seed{}_tl{}'.format(train_seed, min_track_length)

#### Instantiate the tracking model

In [20]:
from deepcell import model_zoo

tracking_model = model_zoo.siamese_model(
    input_shape=in_shape,
    neighborhood_scale_size=neighborhood_scale_size,
    features=features)

#### Train a new tracking model

In [ ]:
from deepcell.training import train_model_siamese_daughter

tracking_model = train_model_siamese_daughter(
    model=tracking_model,
    dataset=DATA_FILE,  # full path to trks file
    model_name=tracking_model_name,
    optimizer=optimizer,
    batch_size=batch_size,
    crop_dim=crop_dim,
    min_track_length=min_track_length,
    features=features,
    neighborhood_scale_size=neighborhood_scale_size,
    test_size=test_size,
    n_epoch=n_epoch,
    model_dir=MODEL_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    class_weight=None,
    seed=train_seed)

training on dataset: /root/.keras/datasets/combined_training_data.trks
saving model at: /data/models/tracking_model_1_tl9.h5
saving loss at: /data/models/tracking_model_1_tl9.npz
X_train shape: (577, 40, 216, 256, 1)
y_train shape: (577, 40, 216, 256, 1)
X_test shape: (145, 40, 216, 256, 1)
y_test shape: (145, 40, 216, 256, 1)
Output Shape: (None, 3)
Training on 1 GPUs
Using real-time data augmentation.


/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)


total_train_pairs: 708688.0
total_test_pairs: 203512.0
batch size: 128
validation_steps:  1589.0
Epoch 1/10
5535/5536 [============================>.] - ETA: 0s - loss: 0.0607 - acc: 0.9826
Epoch 00001: val_loss improved from inf to 0.09630, saving model to /data/models/tracking_model_1_tl9.h5
5536/5536 [==============================] - 4391s 793ms/step - loss: 0.0607 - acc: 0.9826 - val_loss: 0.0963 - val_acc: 0.9781
Epoch 2/10
5535/5536 [============================>.] - ETA: 0s - loss: 0.0282 - acc: 0.9948
Epoch 00002: val_loss did not improve from 0.09630
5536/5536 [==============================] - 4375s 790ms/step - loss: 0.0282 - acc: 0.9948 - val_loss: 0.0967 - val_acc: 0.9785
Epoch 3/10
5535/5536 [============================>.] - ETA: 0s - loss: 0.0231 - acc: 0.9965
Epoch 00003: val_loss did not improve from 0.09630
5536/5536 [==============================] - 4799s 867ms/step - loss: 0.0231 - acc: 0.9965 - val_loss: 0.1174 - val_acc: 0.9760
Epoch 4/10
3385/5536 [===========

#### Load an Existing Model (Optional)

In [2]:
from deepcell import model_zoo

# Tracking model settings
features = {'appearance', 'distance', 'neighborhood', 'regionprop'}
min_track_length = 9
neighborhood_scale_size=30
batch_size = 128  
crop_dim = 32
in_shape = (crop_dim, crop_dim, 1)

# Re-instantiate the tracking model
tracking_model = model_zoo.siamese_model(
    input_shape=in_shape,
    neighborhood_scale_size=neighborhood_scale_size,
    features=features)

# Load model weights
siamese_weights_file = 'tracking_model_seed1_tl9.h5'
siamese_weights_file = os.path.join(MODEL_DIR, siamese_weights_file)

tracking_model.load_weights(siamese_weights_file)

### Investigate Model Performance with a Confusion Matrix - Requires a Seed Value

In [22]:
# Using DATA_FILE from above to extract Test Data

# Import Statements
import deepcell.image_generators as generators
from deepcell.utils.data_utils import get_data

# Get the data
train_dict, test_dict = get_data(DATA_FILE, mode='siamese_daughters', seed=train_seed)

# Build the generator and iterator
datagen_test = generators.SiameseDataGenerator(
        rotation_range=180,  # randomly rotate images by 0 to rotation_range degrees
        shear_range=0,     # randomly shear images in the range (radians , -shear_range to shear_range)
        horizontal_flip=1, # randomly flip images
        vertical_flip=1)   # randomly flip images

test_iterator = generators.SiameseIterator(test_dict,
                                           datagen_test,
                                           neighborhood_scale_size=neighborhood_scale_size,
                                           min_track_length=min_track_length,
                                           features=features)

/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)


In [23]:
from sklearn.metrics import confusion_matrix

Y = []
Y_pred = []
for i in range(1,1000):
    if i % 100 == 0:
        print(".", end="")
    lst, y_true = next(test_iterator)
    y_true = np.argmax(y_true, axis=-1)
    y_pred = np.argmax(tracking_model.predict(lst), axis=-1)
    Y.append(y_true)
    Y_pred.append(y_pred)
    
Y = np.concatenate(Y, axis=0)
Y_pred = np.concatenate(Y_pred, axis=0)

print("")
cm = confusion_matrix(Y, Y_pred)
print(cm)

.........
[[10544     8     8]
 [   21 10484    32]
 [  353   714  9544]]


In [24]:
test_acc = sum(np.array(Y) == np.array(Y_pred)) / len(Y)
print('Accuracy across all three classes: ', test_acc)

# Normalize the diagonal entries of the confusion matrix
cm = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
# Diagonal entries are the accuracies of each class
print('Accuracy for each individual class [Different, Same, Daughter]: ', cm.diagonal())

Accuracy across all three classes:  0.9641730793490602
Accuracy for each individual class [Different, Same, Daughter]:  [0.99848485 0.99497011 0.89944397]


---
## Next Steps

This model is used within an assignment problem framework to track cells through time-lapse sequences and build cell lineages. To see how this works on example data, refer to Part 2 of this notebook series: [Tracking Example with Benchmarking](https://github.com/vanvalenlab/deepcell-tf/blob/master/scripts/tracking/Tracking%20Example%20with%20Benchmarking.ipynb).